In [1]:
import tensorflow as tf
import os

model_dirs = [
    os.path.join(os.curdir, "model", name)
    for name in sorted(os.listdir(os.path.join(os.curdir, "model")))
]
model_dirs


2022-11-17 20:03:46.112024: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 20:03:46.311488: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 20:03:47.226495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-17 20:03:47.226593: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

['./model/BASEL',
 './model/BUDAPEST',
 './model/DE',
 './model/DRESDEN',
 './model/DUSSELDORF',
 './model/HEATHROW',
 './model/KASSEL',
 './model/LJUBLJANA',
 './model/MAASTRICHT',
 './model/MALMO',
 './model/MONTELIMAR',
 './model/MUENCHEN',
 './model/OSLO',
 './model/PERPIGNAN',
 './model/ROMA',
 './model/SONNBLICK',
 './model/STOCKHOLM',
 './model/TOURS']

In [2]:
models = [tf.keras.models.load_model(model_dir) for model_dir in model_dirs]
models[1].summary()

2022-11-17 20:03:48.221776: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:42:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-17 20:03:48.240951: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:42:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-17 20:03:48.241981: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:42:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-17 20:03:48.242768: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Model: "BUDAPEST_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 6, 8)]            0         
                                                                 
 lstm_2 (LSTM)               (None, 6, 16)             1600      
                                                                 
 lstm_3 (LSTM)               (None, 16)                2112      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3,729
Trainable params: 3,729
Non-trainable params: 0
_________________________________________________________________


In [3]:
import pandas as pd

CSV_DIR = os.path.join(os.curdir, "dataset", "sorted")
csv_filenames = sorted(os.listdir(CSV_DIR))
csv_dataset = [
    pd.read_csv(os.path.join(CSV_DIR, filename)) for filename in csv_filenames
]
print(len(csv_dataset))

18


In [4]:

TRAIN_END_INDEX = 2922
VALIDATION_END_INDEX = 3288



def get_dataset(
    dataset: pd.DataFrame,
    sequence_length,
    batch_size,
    train_end_index,
    validation_end_index,
):
    data = dataset
    for col in dataset.columns:
        if "temp_mean" in col:
            target = pd.DataFrame(dataset[col])
            data = data.drop(columns=["DATE"])
            break
    scaled_data = (data - data.mean(axis=0)) / data.std(axis=0)
    train_dataset = tf.keras.utils.timeseries_dataset_from_array(
        scaled_data[:-sequence_length],
        targets=target[sequence_length:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        start_index=0,
        end_index=train_end_index,
    )
    validation_dataset = tf.keras.utils.timeseries_dataset_from_array(
        scaled_data[:-sequence_length],
        targets=target[sequence_length:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        start_index=train_end_index,
        end_index=validation_end_index,
    )
    test_dataset = tf.keras.utils.timeseries_dataset_from_array(
        scaled_data[:-sequence_length],
        targets=target[sequence_length:],
        sequence_length=sequence_length,
        batch_size=batch_size,
        start_index=validation_end_index,
    )
    return (train_dataset, validation_dataset, test_dataset)


sequence_length = 6

batch_dataset = [
    get_dataset(csv_data, sequence_length, 32, TRAIN_END_INDEX, VALIDATION_END_INDEX)
    for csv_data in csv_dataset
]

print(len(batch_dataset))
print(batch_dataset[1])
for data, label in batch_dataset[0][0]:
    print(data[0])
    print(label[0])
    break


18
(<BatchDataset element_spec=(TensorSpec(shape=(None, None, 8), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>, <BatchDataset element_spec=(TensorSpec(shape=(None, None, 8), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>, <BatchDataset element_spec=(TensorSpec(shape=(None, None, 8), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>)
tf.Tensor(
[[ 1.11010855  1.34424888  1.34682202 -1.20851298 -0.38199123 -1.07646018
  -1.09549112 -0.80998751 -1.33429094]
 [ 1.11010855  1.15869866  1.7487146  -1.15505695 -0.43793346 -1.07646018
  -1.00108475 -0.64465741 -1.23109558]
 [-0.17993809  0.602048    1.69847803 -0.8877768  -0.43793346 -0.22197883
  -1.18989749 -1.03543765 -1.23109558]
 [ 0.680093    0.41649778  1.04540259 -0.74879112  0.2147259   0.51703206
  -0.96062487 -0.97531762 -0.92150951]
 [-0.17993809  1.43702399  0.8444563  -0.87708559 -0.30740159 -0.22197883
  -

In [5]:
model_losses = dict()
for model, batch_data in zip(models, batch_dataset):
    loss, mae = model.evaluate(batch_data[2])
    model_losses[model.name] = {"loss": loss, "MAE": mae}
model_losses

2022-11-17 20:05:36.345274: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


12/12 [==============================] - 1s 14ms/step - loss: 4.0815 - MAE: 1.6223


{'BASEL_model': {'loss': 3.9856207370758057, 'MAE': 1.5322635173797607},
 'BUDAPEST_model': {'loss': 3.8022818565368652, 'MAE': 1.49616539478302},
 'DE_model': {'loss': 3.146448850631714, 'MAE': 1.412664532661438},
 'DRESDEN_model': {'loss': 4.31781005859375, 'MAE': 1.6209954023361206},
 'DUSSELDORF_model': {'loss': 3.885239839553833, 'MAE': 1.5584055185317993},
 'HEATHROW_model': {'loss': 1.5891635417938232, 'MAE': 0.9812320470809937},
 'KASSEL_model': {'loss': 3.940459966659546, 'MAE': 1.5299921035766602},
 'LJUBLJANA_model': {'loss': 4.642023086547852, 'MAE': 1.638442039489746},
 'MAASTRICHT_model': {'loss': 4.363713264465332, 'MAE': 1.6281614303588867},
 'MALMO_model': {'loss': 3.0827739238739014, 'MAE': 1.348146915435791},
 'MONTELIMAR_model': {'loss': 3.4046170711517334, 'MAE': 1.4872580766677856},
 'MUENCHEN_model': {'loss': 4.702232837677002, 'MAE': 1.6908601522445679},
 'OSLO_model': {'loss': 3.2046892642974854, 'MAE': 1.3807661533355713},
 'PERPIGNAN_model': {'loss': 4.267011

In [6]:
model_loss_df = pd.DataFrame(model_losses)
model_loss_df

,BASEL_model,BUDAPEST_model,DE_model,DRESDEN_model,DUSSELDORF_model,HEATHROW_model,KASSEL_model,LJUBLJANA_model,MAASTRICHT_model,MALMO_model,MONTELIMAR_model,MUENCHEN_model,OSLO_model,PERPIGNAN_model,ROMA_model,SONNBLICK_model,STOCKHOLM_model,TOURS_model
loss,3.985621,3.802282,3.146449,4.317810,3.885240,1.589164,3.940460,4.642023,4.363713,3.082774,3.404617,4.702233,3.204689,4.267012,2.898797,4.254802,3.075783,4.081547
MAE,1.532264,1.496165,1.412665,1.620995,1.558406,0.981232,1.529992,1.638442,1.628161,1.348147,1.487258,1.690860,1.380766,1.616821,1.268395,1.594640,1.353098,1.622262


In [7]:
model_loss_df.mean(axis=1)

loss    3.702501
MAE     1.486698
dtype: float64